In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import openai
import re
import time
from scripts.prepare_data_helpers import prepare_examples, create_query
import spacy
import textacy
from dotenv import load_dotenv

load_dotenv()
nlp = spacy.load('en_core_web_sm')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB

/Users/marcbraun/.conda/envs/732A81-Text_Mining/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare Data

In [2]:
train1 = pd.read_csv('../../e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../../e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
dev = pd.read_csv('../../e-SNLI/dataset/esnli_dev.csv')
test = pd.read_csv('../../e-SNLI/dataset/esnli_test.csv')

train = train.dropna(subset=['Sentence1', 'Sentence2', 'Explanation_1'])
train = train.reset_index(drop=True)

np.random.seed(12345) # seed for numpy package
test_indices = list(np.random.choice(test.index.values, size=1000, replace=False))
test = test.loc[test_indices]
test = test.reset_index(drop=True)

In [3]:
gpt_pred = pd.read_csv('../output_data/svo_structure_4.csv', sep=';')
# Transform list to single string
gpt_pred.reduced_expl = [i.replace("'", " ").replace(" ", "").strip("][").replace(",", " ") for i in gpt_pred.reduced_expl]
gpt_pred.pos_expl = [i.replace("'", " ").replace(" ", "").strip("][").replace(",", " ") for i in gpt_pred.pos_expl]

In [4]:
gpt_pred.pos_expl

0      S1_ADJ S1S2_NOUN S1S2_ADJ S1S2_NOUN be rephras...
1      S1S2_NOUN be S1_NOUN S1S2_VERB S1_NOUN not S2_...
2      statement imply S1S2_NOUN be S1S2_VERB S1_VERB...
3      be people S2_ADJ S1_NOUN S1_VERB S1_ADJ S1_NOU...
4                S1_VERB S1_NOUN be form S2_VERB S2_NOUN
                             ...                        
995    statement describe S1_NOUN S1_NOUN S1_VERB sta...
996    answer imply S2_NOUN S1S2_NOUN be S1_ADJ S1_NO...
997    clothing not necessarily indicate be S2_VERB S...
998    S1S2_VERB S1S2_NOUN be specific type activity ...
999    S1_NOUN S1_VERB S1_NOUN imply be evening not S...
Name: pos_expl, Length: 1000, dtype: object

# Predict from Reduced Explanations

## SVO Structure

In [5]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(gpt_pred.reduced_expl)

In [6]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), pd.Series([x.lower() for x in gpt_pred.pred_label]), test_size=0.5, random_state=12345)

In [8]:
y_test.value_counts() / y_test.shape[0]

entailment       0.490
contradiction    0.338
neutral          0.172
dtype: float64

In [9]:
mnb = MultinomialNB()
y_pred = mnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of in total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
print(f"Accuracy: {(y_test == y_pred).mean()}")

Number of mislabeled points out of in total 500 points : 111
Accuracy: 0.778


In [10]:
metrics.confusion_matrix(y_test, y_pred)

array([[123,  37,   9],
       [ 21, 218,   6],
       [ 17,  21,  48]])

In [11]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[0,:]) - np.exp(mnb.feature_log_prob_[1,:])) / np.exp(mnb.feature_log_prob_[1,:])).argsort()[::-1]]

array(['be different', 'not', 's1s2_subj not', ..., 's1s2_subj s2_verb',
       'imply s2_subj s2_verb', 'imply s1s2_subj'], dtype=object)

In [12]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[0,:]) - np.exp(mnb.feature_log_prob_[2,:])) / np.exp(mnb.feature_log_prob_[2,:])).argsort()[::-1]]

array(['same', 'be not', 's1_verb s2_verb', ..., 'not mean s2_verb',
       'not mean', 'mean'], dtype=object)

In [13]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[1,:]) - np.exp(mnb.feature_log_prob_[2,:])) / np.exp(mnb.feature_log_prob_[1,:])).argsort()[::-1]]

array(['same', 's1_obj be', 'imply s1s2_subj', ..., 's1_verb not', 'not',
       'not mean'], dtype=object)

In [14]:
y_train.value_counts() / y_train.shape[0]

entailment       0.476
contradiction    0.366
neutral          0.158
dtype: float64

In [15]:
np.exp(mnb.class_log_prior_)

array([0.366, 0.476, 0.158])

In [16]:
cond_prob_Xi_given_not_yj = np.zeros(mnb.feature_log_prob_.shape)
for j in range(3):
    log_joint_prob_m = mnb.feature_log_prob_[j != np.array([0,1,2]),:][0,:] + mnb.class_log_prior_[j != np.array([0,1,2])][0]
    log_joint_prob_k = mnb.feature_log_prob_[j != np.array([0,1,2]),:][1,:] + mnb.class_log_prior_[j != np.array([0,1,2])][1]
    joint_prob_not_j = np.exp(log_joint_prob_m) + np.exp(log_joint_prob_k)
    cond_prob_Xi_given_not_yj[j,:] = joint_prob_not_j / (1 - np.exp(mnb.class_log_prior_[j]))
prob_increase = (np.exp(mnb.feature_log_prob_) - cond_prob_Xi_given_not_yj) / cond_prob_Xi_given_not_yj

In [17]:
prob_increase.argsort()[::-1]

array([[1644, 1056,  779, ...,  435,  423,  559],
       [ 461,   24, 1162, ..., 1307,  325,  320],
       [ 320, 1259,  340, ...,   39,  621,   24]])

In [18]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[0,::-1]]

array(['be different', 'not same', 'be not same', ...,
       'imply s2_subj s2_verb', 's1s2_subj s2_verb', 'imply s1s2_subj'],
      dtype=object)

In [19]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[1,::-1]]

array(['imply s1s2_subj', 'imply s1s2_subj s1s2_verb', 's1s2_verb imply',
       ..., 's1s2_subj not', 'be different', 'not'], dtype=object)

In [20]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[2,::-1]]

array(['not mean', 'mean', 'mean s2_verb', ..., 's1_obj be',
       's1_verb s2_verb', 'same'], dtype=object)

## POS Tags

In [21]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(gpt_pred.pos_expl)

In [22]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), pd.Series([x.lower() for x in gpt_pred.pred_label]), test_size=0.3, random_state=12345)

In [24]:
y_test.value_counts() / y_test.shape[0]

entailment       0.490000
contradiction    0.336667
neutral          0.173333
dtype: float64

In [25]:
mnb = MultinomialNB()
y_pred = mnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of in total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
print(f"Accuracy: {(y_test == y_pred).mean()}")

Number of mislabeled points out of in total 300 points : 42
Accuracy: 0.86


In [26]:
metrics.confusion_matrix(y_test, y_pred)

array([[ 87,  11,   3],
       [  8, 135,   4],
       [ 11,   5,  36]])

In [27]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[0,:]) - np.exp(mnb.feature_log_prob_[1,:])) / np.exp(mnb.feature_log_prob_[1,:])).argsort()[::-1]]

array(['time', 'be different', 's1s2_noun not', ..., 's1s2_noun s1s2_adj',
       'be type', 's1s2_noun imply'], dtype=object)

In [28]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[0,:]) - np.exp(mnb.feature_log_prob_[2,:])) / np.exp(mnb.feature_log_prob_[2,:])).argsort()[::-1]]

array(['time', 's2_noun time', 'different', ..., 'know', 'not know',
       'not mean'], dtype=object)

In [29]:
vectorizer.get_feature_names_out()[((np.exp(mnb.feature_log_prob_[1,:]) - np.exp(mnb.feature_log_prob_[2,:])) / np.exp(mnb.feature_log_prob_[1,:])).argsort()[::-1]]

array(['s1_noun imply', 's1s2_noun s1s2_verb', 'statement imply', ...,
       'not necessarily', 'necessarily', 'mean be'], dtype=object)

In [30]:
y_train.value_counts() / y_train.shape[0]

entailment       0.480000
contradiction    0.358571
neutral          0.161429
dtype: float64

In [31]:
np.exp(mnb.class_log_prior_)

array([0.35857143, 0.48      , 0.16142857])

In [32]:
cond_prob_Xi_given_not_yj = np.zeros(mnb.feature_log_prob_.shape)
for j in range(3):
    log_joint_prob_m = mnb.feature_log_prob_[j != np.array([0,1,2]),:][0,:] + mnb.class_log_prior_[j != np.array([0,1,2])][0]
    log_joint_prob_k = mnb.feature_log_prob_[j != np.array([0,1,2]),:][1,:] + mnb.class_log_prior_[j != np.array([0,1,2])][1]
    joint_prob_not_j = np.exp(log_joint_prob_m) + np.exp(log_joint_prob_k)
    cond_prob_Xi_given_not_yj[j,:] = joint_prob_not_j / (1 - np.exp(mnb.class_log_prior_[j]))
prob_increase = (np.exp(mnb.feature_log_prob_) - cond_prob_Xi_given_not_yj) / cond_prob_Xi_given_not_yj

In [33]:
prob_increase.argsort()[0,::-1]

array([1418,   72, 1177, ...,  171, 1046, 1028])

In [34]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[0,::-1]]

array(['time', 'be different', 's2_noun time', ..., 'be type',
       's1s2_noun s1s2_adj', 's1s2_noun imply'], dtype=object)

In [40]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[1,::-1]]

array(['s1s2_noun imply', 's1_noun imply', 'be type', ..., 'be different',
       'time', 's1s2_noun not'], dtype=object)

In [41]:
vectorizer.get_feature_names_out()[prob_increase.argsort()[2,::-1]]

array(['not mean', 'not know', 'know', ..., 's1s2_noun s1s2_verb',
       's1_noun imply', 'time'], dtype=object)

In [35]:
prob_increase.argsort()[0,::-1]

array([1418,   72, 1177, ...,  171, 1046, 1028])

In [36]:
cond_prob_Xi_given_not_yj[0,1418]

0.00018956571264920502

In [37]:
np.exp(mnb.feature_log_prob_)[0,1418]

0.010141987829614608

In [38]:
np.mean(X_train[y_train == 'contradiction'][:,1418])

0.2151394422310757

In [39]:
vectorizer.get_feature_names_out()[24]

'age s1s2_noun'